In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
df = pd.read_excel("data.xlsx")
df.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [36]:
df.drop(["Date", "Time"], axis=1, inplace=True)
df.info(), df.isnull().sum().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8358 entries, 0 to 8357
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CO(GT)         8358 non-null   float64
 1   PT08.S1(CO)    8358 non-null   float64
 2   NMHC(GT)       8358 non-null   int64  
 3   C6H6(GT)       8358 non-null   float64
 4   PT08.S2(NMHC)  8358 non-null   float64
 5   NOx(GT)        8358 non-null   float64
 6   PT08.S3(NOx)   8358 non-null   float64
 7   NO2(GT)        8358 non-null   float64
 8   PT08.S4(NO2)   8358 non-null   float64
 9   PT08.S5(O3)    8358 non-null   float64
 10  T              8358 non-null   float64
 11  RH             8358 non-null   float64
 12  AH             8358 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 849.0 KB


(None, 0)

In [ ]:
df.fillna(df.mean(), inplace=True)

#### data中没有缺失值，不用对缺失值进行处理

In [37]:
df["COthreshold"] = np.where(df["CO(GT)"] > df["CO(GT)"].mean(), 1, 0)
df["COthreshold"].value_counts()

COthreshold
1    6773
0    1585
Name: count, dtype: int64

#### 目前可以看出，正负样本不均衡，之后要进行样本平衡的处理

##### 要进行两个神经网络的训练，分别求出classfication和regression任务

In [38]:
df.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,COthreshold
0,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754,1
1,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487,1
2,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239,1
3,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713,1
4,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794,1
